In [3]:
from bs4 import BeautifulSoup as beau
import pandas as pd
import itertools
import pickle
import nltk
import os
import re

In [4]:
directory = 'natfinder/natfinder/manual-works'
fics_df = pd.DataFrame(columns=['L1', 'author', 'title', 'tags', 'summary', 'notes', 'endnotes', 'work', 'rating', 'warnings', 'category', 'fandom', 'ships', 'charas', 'freeform'])

In [5]:
def extract_info(soup):
    # basic info
    try:
        L1 = soup.find(class_='L1').text
    except:
        L1 = 'ERROR'
    try:
        author = soup.find(rel='author').text
    except:
        author = 'Anonymous'
    try:
        title = soup.find('h1').text
    except:
        title = 'NaN'
    try:
        tags = [t.text for t in soup.find_all(class_='tag')]
    except:
        tags = 'NaN'
    try:
        summary = soup.find('div', class_='summary module').text
    except:
        summary = 'NaN'
    try:
        notes = soup.find('div', class_='notes module').text
    except:
        notes = 'NaN'
    try:
        endnotes = soup.find('div', class_='end notes module').text
    except:
        endnotes = 'NaN'
    try:
        work = str([x.text for x in soup.find_all(id="chapters")])
    except:
        work = 'NaN'
    # tag info
    try:
        rating = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='rating tags')]
    except:
        rating = 'ERROR'
    try:
        warnings = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='warning tags')]
    except:
        warnings = 'ERROR'
    try:
        categories = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='category tags')]
    except:
        categories = 'ERROR'
    try:
        fandoms = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='fandom tags')]
    except:
        fandoms = 'ERROR'
    try:
        ships = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='relationship tags')]
    except:
        ships = 'NaN'
    try:
        charas = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='character tags')]
    except:
        charas = 'NaN'
    try:
        freeform = [[a.text for a in t.find_all('a')] for t in soup.find_all(class_='freeform tags')]
    except:
        freeform = 'NaN'
    temp = pd.DataFrame({'L1':[L1], 'author':[author], 'title':[title],'tags':[tags], 'summary':[summary], 'notes':[notes], 'endnotes':[endnotes], 'work':[work], 'rating':[rating], 'warnings':[warnings], 'category':[categories], 'fandom':[fandoms], 'ships':[ships], 'charas':[charas], 'freeform':[freeform]})
    return temp

In [6]:
files = []
i = 0
for f in os.scandir(directory):
    if f.name.endswith('.html'):
        files.append(f.name)
for f in files:
    soup = beau(open(directory + '/' + f).read(), 'html.parser')
    temp_df = extract_info(soup)
    fics_df = pd.concat([fics_df, temp_df], ignore_index=True)

In [7]:
fics_df['work'] = fics_df.work.map(lambda x : re.sub(r'(?s).*Work Text:', '', re.sub(r'(?s)\xa0\n+.*', '', re.sub(r'\s', ' ', x))))

In [8]:
with open('fics_df_manual.pkl', 'wb') as file:
    pickle.dump(fics_df, file)